In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
rm -rf /kaggle/working/ChatGLM3

### git clone 自己的chatglm3 new11分支

In [ ]:
%cd /kaggle/working
!git clone -b new11 https://github.com/valkryhx/ChatGLM3

In [ ]:
%cd /kaggle/working/ChatGLM3
!git status

In [ ]:
%cd /kaggle/working/ChatGLM3
!git pull --all --force
!pip install -r requirements.txt

In [ ]:
!pip install peft==0.6.2
!pip install bitsandbytes==0.41.2
!pip install accelerate==0.24.0
# !pip install peft==0.5.0
# !pip install accelerate==0.21.0
# !pip install bitsandbytes==0.41.0

In [ ]:
!pip install deepspeed

In [ ]:
%pwd
!ls -lrt

In [ ]:
#rm -rf /kaggle/working/formatted_data

### 进入到finetune_chatmodel_demo目录
### 将toolAlpaca格式处理成chatglm3 多轮对话所需格式 结果
### 在/kaggle/working/ChatGLM3/finetune_chatmodel_demo/formatted_data/tool_alpaca.jsonl

In [ ]:
%cd /kaggle/working/ChatGLM3/finetune_chatmodel_demo

In [ ]:
!python /kaggle/working/ChatGLM3/finetune_chatmodel_demo/scripts/format_tool_alpaca.py --path /kaggle/working/ChatGLM3/finetune_chatmodel_demo/ToolAlpaca/data/train_data.json

In [ ]:
#ls /kaggle/working/ChatGLM3/finetune_chatmodel_demo/formatted_data

### 加载tool_alpaca.jsonl文件为 train_data 变量

In [ ]:
import json
train_file = "/kaggle/working/ChatGLM3/finetune_chatmodel_demo/formatted_data/tool_alpaca.jsonl"
with open(train_file, "r", encoding="utf-8") as f:
    if train_file.endswith(".json"):
        train_data = json.load(f)
    elif train_file.endswith(".jsonl"):
        train_data = [json.loads(line) for line in f]

In [ ]:
print(train_data[:2])

In [ ]:
pwd

### import 各类依赖

In [ ]:
import logging
import os
import sys
import torch
import json
import transformers
from transformers import (
    AutoConfig,
    AutoModel,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    Seq2SeqTrainingArguments,
    set_seed,
)
from trainer import PrefixTrainer

from arguments import ModelArguments, DataTrainingArguments

from preprocess_utils import sanity_check, MultiTurnDataset, InputOutputDataset

logger = logging.getLogger(__name__)

### 加载chatglm3-6b的tokenizer

In [ ]:
BASE_MODEL_PATH="THUDM/chatglm3-6b"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True)

### 我们先看看这一版的tokenizer的特殊token有哪些 发现 <|system|> <|user|> <|assistanta|> 和 <|observation|>都是特殊token，不会被一个字符一个字符的解析


In [ ]:
from loguru import logger
logger.info(tokenizer.tokenizer.index_special_tokens)

### 将list[dict]格式的train_data 转换成dict格式，包含input_ids 和label 两个key 

In [ ]:
max_seq_length =2048

In [ ]:
train_dataset = MultiTurnDataset(
            train_data,
            tokenizer,
            max_seq_length
        )

### sanity check

In [ ]:
sanity_check(train_dataset[2]['input_ids'], train_dataset[2]['labels'], tokenizer)

In [ ]:
#!git pull --force

In [ ]:
#cat /kaggle/working/ChatGLM3/finetune_chatmodel_demo/preprocess_utils.py

In [ ]:
print(len(train_dataset))

In [ ]:
!git pull --force
!ls -lrt /kaggle/working/ChatGLM3/finetune_chatmodel_demo/formatted_data

In [ ]:
!git pull --force
!deepspeed --include localhost:0,1  hx_sft_multi_turn_func_call.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm3-6b \
  --output_dir func_call_sft_v1 \
  --num_train_samples -1 \
  --num_eval_samples 2 \
  --train_data_path /kaggle/working/ChatGLM3/finetune_chatmodel_demo/formatted_data/tool_alpaca.jsonl  \
  --eval_data_path  /kaggle/working/ChatGLM3/finetune_chatmodel_demo/formatted_data/tool_alpaca.jsonl     \
  --data_type sharegpt  \
  --max_length 600 \
  --lora_rank 8 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 2  \
  --gradient_accumulation_steps 1 \
  --learning_rate  1.8e-5 \
  --num_train_epochs  40  \
  --save_total_limit 2 \
  --load_in_4bit True \
--deepspeed ds_zero2_config.json